# Tutorial 1) Parameterizing sequences with arbok

## Introduction

This tutorial gives a first insight into the architecture and philosophy behind arbok.
In the first step, a  `Sample` object will be configured and is used in every following
sequence.
Paramerizing of a sequence is demonstrated by a simple square pulse.

Firstly the `ArbokDriver` itself, the generic `Sample` and `Sequence` class is imported.
We also import a dummy configuration which has been taken from the quantum machines github repository

In [1]:
from arbok_driver import ArbokDriver, Sample, Sequence
from example_sequences.configuration import qm_config

2024-04-06 16:52:59,331 - qm - INFO     - Starting session: 12d93ac4-0432-49b4-92a3-982cce74004f


## Configuring a `Sample`

The sample holds the configuration of the quantum machine that you probably already have from your experimetns.

Besides this configuration a further 'divider_config' is required.
This dictionairy represents voltage divider that are in between the quantum machine and your sample.

All voltage values in arbok are meant to be what is applied to the sample, not the output of the machine.
This is implemented by the scale attribute of [qcodes parameters](https://microsoft.github.io/Qcodes/api/parameters/index.html#qcodes.parameters.Parameter).

A simple example is given below. A dict entry is required for every element in the quantum machines config that has an output port configured.

In [2]:
opx_scale = 2
divider_config = {
    'gate_1': {
        'division': 1*opx_scale,
    },
    'gate_2': {
        'division': 1*opx_scale,
    },
    'readout_element': {
        'division': 1*opx_scale
    }
}

Both of those configs are now used to instanciate the given sample.

In [3]:
dummy_sample = Sample('dummy_sample', qm_config, divider_config)

## Building the `Arbok_driver`

The sample we created previously is the only requirement to build a basic arbok_driver.

In [4]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)

`Sequence`s can now be registered into this driver.

Sequences are meant to summarize all sub-sequences that are required to run a single type of measurement.
Currently single sequences per driver are supported.

In [5]:
dummy_sequence = Sequence('dummy_squence', dummy_sample)
qm_driver.add_sequence(dummy_sequence)

## Configuring a simple square pulse sequence

So far, only structural classes were created and assebled with each other.
This changes with the given example of a `SquarePulse`. See the source code below.

In [6]:
from example_sequences.square_pulse import SquarePulse
SquarePulse??

Init signature: SquarePulse(*args: 'Any', **kwargs: 'Any') -> 'Any'
Source:        
class SquarePulse(SubSequence):
    """
    Class containing parameters and sequence for a simple square pulse
    """
    def __init__(self, name: str,sample: Sample, seq_config: dict):
        """
        Constructor method for 'SquarePulse' class
        
        Args:
            name (str): name of sequence
            sample  (Sample): Sample class for physical device
            config (dict): config containing pulse parameters
        """
        super().__init__(name, sample, seq_config)

    def qua_sequence(self):
        """Macro that will be played within the qua.program() context"""
        qua.align()
        qua.play('ramp'*qua.amp(self.amplitude()), self.element())
        qua.wait(self.t_square_pulse(), self.element())
        qua.play('ramp'*qua.amp(-self.amplitude()), self.element())
File:           c:\users\labuser\gitrepos\arbok_driver\docs\example_sequences\square_pulse.py
Type:  

While looking at the source code above you might notice that there are attributes being called that are not created in the classes `__init__` method (e.g self.amplitude, self.element, etc.)
Those are created from a given `sequence_config` file that adds a `qcodes.parameter` for each entry like the one given here:

In [7]:
square_conf = {
    'amplitude': {
        'value': 0.5,
        'unit': 'V',
    },
    't_square_pulse': {
        'value': 100,
        'unit': 'cycles'
    },
    'element': {
        'value': 'gate_1',
        'unit': 'gate label'
    }
}

With this config we fully configure those parameters with an initial value and
unit. Optionally you can add the variable type within qua, an axis label for data savining and validators like this:

In [8]:
from qcodes.validators import Ints
_ = {
    't_square_pulse': {
        'value': 100,
        'unit': 'cycles',
        'label': 'Square pulse width',
        'qua_type': int,
        'validators': Ints(min_value=4)
    },
}

After creating the square pulse `SubSequence` with the respective config we can take a look at its snapshot.

In [9]:
square_pulse = SquarePulse('square_pulse', dummy_sample, square_conf)
dummy_sequence.add_subsequence(square_pulse)

Again, as all (sub-) sequences so far it requires a sample object for instanciation.
This makes sure that all added sub-sequences are configured for the same device.

In [10]:
qm_driver.dummy_squence.square_pulse.print_readable_snapshot()

square_pulse:
	parameter     value
--------------------------------------------------------------------------------
IDN            :	None 
amplitude      :	0.5 (V)
element        :	gate_1 (gate label)
t_square_pulse :	100 (cycles)


In [11]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    'compiled_qua_program.py', qua_program)
import compiled_qua_program
compiled_qua_program??

Type:        module
String form: <module 'compiled_qua_program' from 'C:\\Users\\labuser\\GitRepos\\arbok_driver\\docs\\compiled_qua_program.py'>
File:        c:\users\labuser\gitrepos\arbok_driver\docs\compiled_qua_program.py
Source:     

# Single QUA script generated at 2024-04-06 16:53:00.234955
# QUA library version: 1.1.7

from qm.qua import *

with program() as prog:
    with infinite_loop_():
        pause()
        align()
        play("ramp"*amp(0.5), "gate_1")
        wait(100, "gate_1")
        play("ramp"*amp(-0.5), "gate_1")


config = None

loaded_config = None

## Parameter sweeps

In [12]:
import numpy as np

In [13]:
dummy_sequence.set_sweeps(
    {
        square_pulse.amplitude: np.linspace(0.1, 1, 5)
    }
)

In [14]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    'compiled_qua_program.py', qua_program)
import compiled_qua_program
compiled_qua_program??

C:\Users\labuser\GitRepos\arbok_driver\arbok_driver\sweep.py:218: UserWarning: Your input array None for amplitude will be parametrized, check output
  warnings.warn(


Type:        module
String form: <module 'compiled_qua_program' from 'C:\\Users\\labuser\\GitRepos\\arbok_driver\\docs\\compiled_qua_program.py'>
File:        c:\users\labuser\gitrepos\arbok_driver\docs\compiled_qua_program.py
Source:     

# Single QUA script generated at 2024-04-06 16:53:00.317319
# QUA library version: 1.1.7

from qm.qua import *

with program() as prog:
    v1 = declare(int, )
    v2 = declare(int, )
    with infinite_loop_():
        pause()
        with for_(v1,0,(v1<1.125),(v1+0.25)):
            align()
            play("ramp"*amp(v1), "gate_1")
            wait(100, "gate_1")
            play("ramp"*amp((0-v1)), "gate_1")


config = None

loaded_config = None

## Readout sequences

In [15]:
from example_sequences.dummy_readout import DummyReadout

In [16]:
readout_sequence_config = {
    'parameters': {
        't_between_measurements': {
            'value': 50,
            'unit': 'cycles',
        }
    },
    'signals':{
        'qubit1':{
            'elements': {
                'sensor1': 'readout_element',
            },
            'readout_points': {
                'ref': {
                    'method': 'average',
                    'desc':'reference point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                },
                'read': {
                    'method': 'average',
                    'desc': 'redout point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                }
            }
        },
    },
    'readout_groups': {
        'difference': {
            'qubit1__diff': {
                'method': 'difference',
                'name': 'diff',
                'args': {
                    'signal': 'qubit1',
                    'minuend': 'qubit1.ref.sensor1_IQ',
                    'subtrahend': 'qubit1.read.sensor1_IQ',
                },
            },
        }
    },
}

In [17]:
dummy_readout = DummyReadout('dummy_readout', dummy_sample, readout_sequence_config)

dict_keys(['dummy_readout_qubit1__ref__sensor1_I', 'dummy_readout_qubit1__ref__sensor1_Q', 'dummy_readout_qubit1__ref__sensor1_IQ'])
dict_keys(['dummy_readout_qubit1__read__sensor1_I', 'dummy_readout_qubit1__read__sensor1_Q', 'dummy_readout_qubit1__read__sensor1_IQ'])


In [18]:
dummy_sequence.add_subsequence(dummy_readout)

In [19]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    'compiled_readout_qua_program.py', qua_program)
import compiled_readout_qua_program
compiled_readout_qua_program??

C:\Users\labuser\GitRepos\arbok_driver\arbok_driver\sweep.py:218: UserWarning: Your input array None for amplitude will be parametrized, check output
  warnings.warn(


Type:        module
String form: <module 'compiled_readout_qua_program' from 'C:\\Users\\labuser\\GitRepos\\arbok_driver\\docs\\compiled_readout_qua_program.py'>
File:        c:\users\labuser\gitrepos\arbok_driver\docs\compiled_readout_qua_program.py
Source:     

# Single QUA script generated at 2024-04-06 16:53:00.523429
# QUA library version: 1.1.7

from qm.qua import *

with program() as prog:
    v1 = declare(int, )
    v2 = declare(fixed, )
    v3 = declare(fixed, )
    v4 = declare(fixed, )
    v5 = declare(fixed, )
    v6 = declare(fixed, )
    v7 = declare(fixed, )
    v8 = declare(fixed, )
    v9 = declare(int, )
    with infinite_loop_():
        pause()
        with for_(v1,0,(v1<1.125),(v1+0.25)):
            align()
            play("ramp"*amp(v1), "gate_1")
            wait(100, "gate_1")
            play("ramp"*amp((0-v1)), "gate_1")
            align()
            measure("measure", "readout_element", None, integration.full("x", v2, ""), integration.full("y", v3, ""))
